In [10]:
import torch 
# from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import RobertaTokenizer, RobertaModel

import os
import pandas as pd
import tensorflow as tf
from datasets import Dataset
from functools import reduce


Roberta
 The inputs of the model take pieces of 512 contiguous tokens that may span over documents

In [11]:
MODEL_NAME = 'roberta-base'


tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
model = RobertaModel.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data_path = os.path.join('..','data', 'raw', 'train.json')

data = pd.read_json(data_path)

data_dataset = Dataset.from_pandas(data)

In [23]:
LABELS_LIST = ['B-NAME_STUDENT', 'B-EMAIL', 'B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM', 'B-URL_PERSONAL', 'B-STREET_ADDRESS', 'I-NAME_STUDENT', 'I-EMAIL', 'I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM','I-URL_PERSONAL','I-STREET_ADDRESS', 'O']

label2id = {label: i for i, label in enumerate(LABELS_LIST)}
label2id['[PAD]'] = -100
id2label = {i: label for label, i in label2id.items()}
id2label

{0: 'B-NAME_STUDENT',
 1: 'B-EMAIL',
 2: 'B-USERNAME',
 3: 'B-ID_NUM',
 4: 'B-PHONE_NUM',
 5: 'B-URL_PERSONAL',
 6: 'B-STREET_ADDRESS',
 7: 'I-NAME_STUDENT',
 8: 'I-EMAIL',
 9: 'I-USERNAME',
 10: 'I-ID_NUM',
 11: 'I-PHONE_NUM',
 12: 'I-URL_PERSONAL',
 13: 'I-STREET_ADDRESS',
 14: 'O',
 -100: '[PAD]'}

In [24]:
def encode_labels(example):
    """
    to be used with datasets.map() with batched=False
    
    Encodes the labels into integers.
    
    """
    labels = example['labels']
    encoded = [label2id[label] for label in labels]
    return {'labels': encoded}

In [25]:
# data_labels_encoded = data_dataset.map(encode_labels)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [50]:
def tokenize_and_align(example, overlap_size = 0):

    org_labels = example['labels']
    tokenized_inputs = tokenizer(example['full_text'][0], is_split_into_words=True, truncation=True, padding='max_length', return_overflowing_tokens=True, stride=overlap_size, return_tensors='pt')
    
    new_labels = []
    org_word_ids_list = []
    document_id = []
    
    #iterating over chunks
    for i, chunk in enumerate(tokenized_inputs['input_ids']):
        ids_of_tokens = tokenized_inputs.word_ids(i)
        
        org_word_ids_list.append(ids_of_tokens)
        document_id.append(example['document'])
        #iterating over ids of tokens
        chunk_labels = []
        for id in ids_of_tokens:
            #if id=None, then it means it's some BERT token (CLS, SEP or PAD)
            if id is None:
                chunk_labels.append(-100)
            else:
                chunk_labels.append(org_labels[id])
        new_labels.append(chunk_labels)

    tokenized_inputs['labels'] = new_labels
    tokenized_inputs['org_word_ids'] = org_word_ids_list
    tokenized_inputs['document'] = document_id

    return tokenized_inputs

In [63]:
example = data_dataset['full_text'][0]
tokenized_inputs = tokenizer(example, truncation=True, padding=True, return_overflowing_tokens=True, return_tensors='pt')
# tokenized_input = tokenizer(example, is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens))
len(tokenized_inputs['input_ids'])


1

In [ ]:
# tokenized_inputs.word_ids(0)
from transformers import PreTrainedTokenizerFast

# Assuming 'tokenizer' is your tokenizer object
if isinstance(tokenizer, PreTrainedTokenizerFast):
    print("This is a fast tokenizer.")
else:
    print("")
